**✅ All-in-One Colab Script: KML ↔ SHP**

The geometry handling is done by "geopandas", for convesion to KML, KML writing done by "simplekml" and for conversion to Shapefile, written with Geopandas ".to_file()", then packaged with "zipfile". When converting Shp to kml, select all shape files and its dependencies.

In [4]:
# STEP 1: Install required packages
!pip install geopandas fiona pyproj

# STEP 2: User selects conversion direction
import ipywidgets as widgets
from IPython.display import display
import geopandas as gpd
from google.colab import files
import os
import zipfile
import shutil

# Helper: Clean up folders/files
def clean_output(path):
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Main conversion logic
def convert_kml_to_shp(kml_file):
    base_name = os.path.splitext(kml_file)[0]
    output_folder = "shapefile_output"
    zip_name = f"{base_name}_shapefile.zip"

    clean_output(output_folder)
    clean_output(zip_name)

    gdf = gpd.read_file(kml_file, driver='KML')
    os.makedirs(output_folder, exist_ok=True)
    gdf.to_file(os.path.join(output_folder, base_name + ".shp"), driver="ESRI Shapefile")

    with zipfile.ZipFile(zip_name, "w") as zipf:
        for file in os.listdir(output_folder):
            zipf.write(os.path.join(output_folder, file), arcname=file)

    files.download(zip_name)

def convert_shp_to_kml(uploaded_files):
    input_folder = "shapefile_input"
    clean_output(input_folder)
    os.makedirs(input_folder, exist_ok=True)

    for fname in uploaded_files.keys():
        os.rename(fname, os.path.join(input_folder, fname))

    shp_file = next((f for f in os.listdir(input_folder) if f.endswith(".shp")), None)
    if not shp_file:
        raise ValueError("No .shp file found.")

    base_name = os.path.splitext(shp_file)[0]
    shp_path = os.path.join(input_folder, shp_file)
    kml_path = f"{base_name}.kml"

    gdf = gpd.read_file(shp_path)
    gdf.to_file(kml_path, driver='KML')

    files.download(kml_path)

# Widget UI
dropdown = widgets.Dropdown(
    options=["Select conversion type", "KML ➝ SHP", "SHP ➝ KML"],
    value="Select conversion type",
    description="Convert:",
    style={'description_width': 'initial'}
)
display(dropdown)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choice = change['new']

        if choice == "KML ➝ SHP":
            print("Upload a .kml file:")
            uploaded = files.upload()
            kml_file = next(iter(uploaded))
            convert_kml_to_shp(kml_file)

        elif choice == "SHP ➝ KML":
            print("Upload all required Shapefile parts (.shp, .shx, .dbf, .prj, etc.):")
            uploaded = files.upload()
            convert_shp_to_kml(uploaded)

dropdown.observe(on_change)


Dropdown(description='Convert:', options=('Select conversion type', 'KML ➝ SHP', 'SHP ➝ KML'), style=Descripti…

**If we want to convert each polyline in SHP TO KML format
**bold text****

---





In [ ]:
# STEP 1: Install libraries
!pip install geopandas simplekml pyshp

# STEP 2: Upload all shapefile parts
from google.colab import files
import os

uploaded = files.upload()  # Upload all .shp, .shx, .dbf, .prj etc.

# Create working folder
input_folder = "/content/shapefiles"
os.makedirs(input_folder, exist_ok=True)

# Move uploaded files
for fname in uploaded.keys():
    os.rename(fname, os.path.join(input_folder, fname))

# STEP 3: Convert each polyline to individual KML files
import geopandas as gpd
import simplekml
import zipfile

output_kml_folder = "/content/kml_outputs"
os.makedirs(output_kml_folder, exist_ok=True)

def export_individual_polylines(shapefile_path, output_dir):
    gdf = gpd.read_file(shapefile_path)
    lines_gdf = gdf[gdf.geometry.type.isin(['LineString', 'MultiLineString'])]

    if lines_gdf.empty:
        print(f"No polylines in {shapefile_path}")
        return

    base_name = os.path.splitext(os.path.basename(shapefile_path))[0]

    for idx, row in lines_gdf.iterrows():
        kml = simplekml.Kml()
        geom = row.geometry
        coords = []

        if geom.geom_type == 'LineString':
            coords = list(geom.coords)
        elif geom.geom_type == 'MultiLineString':
            for part in geom.geoms:  # ✅ FIX HERE
                coords.extend(part.coords)

        line_name = f"{base_name}_{idx+1}"
        linestring = kml.newlinestring(name=line_name)
        linestring.coords = coords

        for field in lines_gdf.columns:
            if field != 'geometry':
                linestring.extendeddata.schemadata.newsimpledata(field, str(row[field]))

        kml_path = os.path.join(output_dir, f"{line_name}.kml")
        kml.save(kml_path)

# STEP 4: Process all .shp files in input folder
for file in os.listdir(input_folder):
    if file.endswith(".shp"):
        shp_path = os.path.join(input_folder, file)
        export_individual_polylines(shp_path, output_kml_folder)

# STEP 5: Zip all individual KMLs
zip_path = "/content/polylines_kmls.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in os.listdir(output_kml_folder):
        zipf.write(os.path.join(output_kml_folder, file), arcname=file)

# STEP 6: Download zipped KMLs
files.download(zip_path)


Saving Bajhang_r.dbf to Bajhang_r.dbf
Saving Bajhang_r.prj to Bajhang_r.prj
Saving Bajhang_r.sbn to Bajhang_r.sbn
Saving Bajhang_r.sbx to Bajhang_r.sbx
Saving Bajhang_r.shp to Bajhang_r.shp
Saving Bajhang_r.shp.xml to Bajhang_r.shp.xml
Saving Bajhang_r.shx to Bajhang_r.shx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>